In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os

In [3]:
import gdown
from zipfile import ZipFile
from pathlib import Path

dataset_url = f"https://drive.google.com/uc?id=1w4DKQ_MMrd0tj31_cJ3YBp1hSMMRMjty"

datafile = f"./data.zip"
if not os.path.exists(datafile):
    gdown.download(url=dataset_url, output=datafile, quiet=False)

with ZipFile(Path(datafile), "r") as zip_ref:
    zip_ref.extractall()

In [4]:
dataframes = []
count=3

for dirname, _, filenames in os.walk('./'):
    for filename in filenames:
        file = os.path.join(dirname, filename)
        if file.endswith(".csv"):
            df = pd.read_csv(file)
            dataframes.append(df.head(20))
            count-=1
            if count == 0 :
                break

# Concatenate all DataFrames
combined_df = pd.concat(dataframes, ignore_index=True)

combined_df = combined_df.drop(columns=['report_name',"product_base_id","report_id"])

# Show the combined DataFrame
print(combined_df.head())

                                        product_name  \
0  ⛅ Kem Chống Nắng Dạng Xịt Skin Aqua Sara-Fit U...   
1  Kem Chống Nắng SPF50 +, Chống Tia UV, Chống Nắ...   
2            Kem chống nắng Klavuu tone up sunscreen   
3  Serum Trắng Da Chống Nắng Vaseline Healthy Whi...   
4  [HCM]Sample Vichy Kem Chống Nắng Ngăn Sạm Da G...   

                                       url_thumbnail  \
0  https://cf.shopee.vn/file/3f87066958412a80a85d...   
1  https://p16-oec-sg.ibyteimg.com/tos-alisg-i-ap...   
2  https://cf.shopee.vn/file/c448f3ccd07b2c13b234...   
3  https://cf.shopee.vn/file/41314c15a3a3760a3775...   
4  https://vn-live-01.slatic.net/p/676a89a3b9b8ad...   

                                                 url original_brand  \
0     https://shopee.vn/product/192924831/4117849404      skin aqua   
1  https://www.tiktok.com/view/product/1729674575...            NaN   
2      https://shopee.vn/product/66709839/8954981233            NaN   
3      https://shopee.vn/product/10464922/

In [5]:
import requests
import os

# Create a folder to store downloaded images
image_folder = 'downloaded_images'
os.makedirs(image_folder, exist_ok=True)

# Function to download an image and return the local file path
def download_image(url, folder):
    try:
        # Extract the image file name from the URL
        image_name = os.path.basename(url)
        
        # Set the local file path
        local_path = os.path.join(folder, image_name)
        
        # Send a GET request to download the image
        response = requests.get(url, stream=True)
        
        print(f'downloading: {url}')
        if response.status_code == 200:
            # Write the image content to the local file
            with open(local_path, 'wb') as f:
                f.write(response.content)
            return local_path
        else:
            return None
    except Exception as e:
        print(f"Error downloading {url}: {e}")
        return None


train_df = combined_df
# Iterate over the DataFrame and replace the image URLs with local paths
train_df['image_paths'] = combined_df['url_thumbnail'].apply(lambda url: download_image(url, image_folder))

# Drop the original 'image_urls' column if desired
train_df = combined_df.drop(columns=['url_thumbnail'])

# Show the DataFrame with the local image paths
print(train_df.head())

downloading: https://cf.shopee.vn/file/3f87066958412a80a85df1fea4b20a29
downloading: https://p16-oec-sg.ibyteimg.com/tos-alisg-i-aphluv4xwc-sg/6b09f77a4f804d93b77d9ea3cced4478~tplv-aphluv4xwc-resize-webp:300:300.webp?from=3170420761
downloading: https://cf.shopee.vn/file/c448f3ccd07b2c13b23403a4d90b995b
downloading: https://cf.shopee.vn/file/41314c15a3a3760a3775ada73c55b87e
downloading: https://vn-live-01.slatic.net/p/676a89a3b9b8adf89680086a90f52361.png
downloading: https://cf.shopee.vn/file/vn-11134207-7r98o-llk4jj5jb4js73
downloading: https://cf.shopee.vn/file/vn-11134207-7qukw-lik2f2i95j4c0b
downloading: https://cf.shopee.vn/file/e519d4f48e952ec04266fb03bf95a8ec
downloading: https://cf.shopee.vn/file/vn-11134201-7qukw-lephtlehcbzm68
downloading: https://cf.shopee.vn/file/16f6e1014be646a25a4476e67d983504
downloading: https://cf.shopee.vn/file/4203ddbb02bf8ac289475d428c5806da
downloading: https://filebroker-cdn.lazada.vn/kf/Scf006b3b20294d1d8a6f732e10eaa211Z.jpg
downloading: https://

In [6]:
from sklearn.model_selection import train_test_split

train_df,test_df = train_test_split(train_df)

In [8]:
from autogluon.multimodal import MultiModalPredictor

predictor = MultiModalPredictor(label="product_group").fit(
    train_data=train_df,
    time_limit=120
)

No path specified. Models will be saved in: "AutogluonModels/ag-20240917_090528"
=================== System Info ===================
AutoGluon Version:  1.1.0
Python Version:     3.10.14
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #14~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Mon Nov 20 18:15:30 UTC 2
CPU Count:          6
Pytorch Version:    2.1.2+cu121
CUDA Version:       12.1
Memory Avail:       16.48 GB / 31.00 GB (53.2%)
Disk Space Avail:   73.76 GB / 1832.70 GB (4.0%)
AutoGluon infers your prediction problem is: 'multiclass' (because dtype of label-column == object).
	3 unique label values:  ['máy hút bụi', 'tẩy da chết', 'kem chống nắng']
	If 'multiclass' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])

AutoMM starts to create your model. ✨✨✨

To track the learning progress, you can open a terminal and launch Tensorboard

/home/minhvv/miniconda3/envs/automl/lib/python3.10/site-packages/lightning/pytorch/loops/fit_loop.py:293: The number of training batches (5) is smaller than the logging interval Trainer(log_every_n_steps=10). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 1:  40%|████      | 2/5 [00:03<00:05,  0.53it/s]

Epoch 1, global step 1: 'val_accuracy' reached 0.44444 (best 0.44444), saving model to '/home/minhvv/ISE_AutoML/ml_service/testing/AutogluonModels/ag-20240917_090528/epoch=1-step=1.ckpt' as top 3


Epoch 2:  40%|████      | 2/5 [00:03<00:04,  0.63it/s]

Epoch 2, global step 2: 'val_accuracy' reached 0.66667 (best 0.66667), saving model to '/home/minhvv/ISE_AutoML/ml_service/testing/AutogluonModels/ag-20240917_090528/epoch=2-step=2.ckpt' as top 3
Time limit reached. Elapsed time is 0:02:28. Signaling Trainer to stop.


Epoch 2:  60%|██████    | 3/5 [01:14<00:49,  0.04it/s]


Start to fuse 2 checkpoints via the greedy soup algorithm.


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00,  3.77it/s]


AutoMM has created your model. 🎉🎉🎉

To load the model, use the code below:
    ```python
    from autogluon.multimodal import MultiModalPredictor
    predictor = MultiModalPredictor.load("/home/minhvv/ISE_AutoML/ml_service/testing/AutogluonModels/ag-20240917_090528")
    ```

If you are not satisfied with the model, try to increase the training time, 
adjust the hyperparameters (https://auto.gluon.ai/stable/tutorials/multimodal/advanced_topics/customization.html),
or post issues on GitHub (https://github.com/autogluon/autogluon/issues).




In [13]:
# scores = predictor.evaluate(test_df, metrics=[
#     "accuracy",
#     "acc",
#     "balanced_accuracy",
#     "mcc",
#     "roc_auc_ovo_macro",
#     "log_loss",
#     "nll",
#     "pac",
#     "pac_score",
#     "quadratic_kappa",
#     "precision_macro",
#     "precision_micro",
#     "precision_weighted",
#     "recall_macro",
#     "recall_micro",
#     "recall_weighted",
#     "f1_macro",
#     "f1_micro",
#     "f1_weighted",
#   ])
scores = predictor.evaluate(test_df, metrics=[
    "accuracy",
    "log_loss",
    "f1_macro",
  ])
scores

Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00,  4.62it/s]


{'accuracy': 0.7333333333333333,
 'log_loss': 18.420680743952364,
 'f1_macro': 0.5833333333333334}